# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,23.08,100,100,2.99,EC,1715508712
1,1,thunder bay,48.4001,-89.3168,12.99,47,75,2.06,CA,1715508712
2,2,forfar,56.6431,-2.8903,17.20,81,100,3.79,GB,1715508713
3,3,adamstown,-25.0660,-130.1015,20.20,65,0,2.67,PN,1715508661
4,4,wailua homesteads,22.0669,-159.3780,21.24,85,75,3.09,US,1715508715


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',  # You can choose any colormap you prefer
    title='City Map with Humidity',
    hover_cols=['City', 'Country', 'Humidity'],
    size='Humidity',  # Size of the point represents the humidity
    frame_width=600,
    frame_height=400
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria
# For example, let's say your ideal weather conditions are:
# - Max temperature lower than 30 degrees but higher than 21
# - Wind speed less than 4.5 m/s
# - Cloudiness less than 40

ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] < 30) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] < 40 )
].copy()  # Create a copy of the filtered DataFrame

# Drop any rows with null values
#ideal_conditions_df.dropna(inplace=True)

# Display sample data
ideal_conditions_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,kingston,17.9970,-76.7936,27.53,83,20,3.09,JM,1715508717
17,17,mchinji,-13.7984,32.8802,27.38,33,0,3.44,MW,1715508722
42,42,solwezi,-12.1688,26.3894,27.82,35,10,3.44,ZM,1715508737
64,64,longido,-2.7333,36.6833,26.07,48,21,4.47,TZ,1715508751
83,83,ongwediva,-17.7833,15.7667,28.87,18,0,3.03,NaN,1715508762


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,kingston,JM,17.9970,-76.7936,83,
17,mchinji,MW,-13.7984,32.8802,33,
42,solwezi,ZM,-12.1688,26.3894,35,
64,longido,TZ,-2.7333,36.6833,48,
83,ongwediva,NaN,-17.7833,15.7667,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter parameter with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kingston - nearest hotel: No hotel found
mchinji - nearest hotel: No hotel found
solwezi - nearest hotel: No hotel found
longido - nearest hotel: Hotel AR Almerimar
ongwediva - nearest hotel: No hotel found
loiza - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: No hotel found
heishan - nearest hotel: Третьяков
karratha - nearest hotel: No hotel found
alofi - nearest hotel: No hotel found
santa ana de yacuma - nearest hotel: No hotel found
taza - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
nueva gerona - nearest hotel: No hotel found
chipinge - nearest hotel: No hotel found
cockburn town - nearest hotel: No hotel found
nchelenge - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
thabazimbi - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
ikalamavony - nearest hotel: No hotel found
lospalos - nearest hotel: Mountshan

,City,Country,Lat,Lng,Humidity,Hotel Name
8,kingston,JM,17.9970,-76.7936,83,No hotel found
17,mchinji,MW,-13.7984,32.8802,33,No hotel found
42,solwezi,ZM,-12.1688,26.3894,35,No hotel found
64,longido,TZ,-2.7333,36.6833,48,Hotel AR Almerimar
83,ongwediva,NaN,-17.7833,15.7667,18,No hotel found
96,loiza,PR,18.4313,-65.8802,87,No hotel found
117,sao joao da barra,BR,-21.6403,-41.0511,85,No hotel found
123,sao felix do xingu,BR,-6.6447,-51.9950,94,No hotel found
143,heishan,CN,41.6892,122.1128,25,Третьяков
162,karratha,AU,-20.7377,116.8463,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
# We'll update the hover_cols parameter to include the "Hotel Name" and "Country" columns.
# Assuming the column names in the hotel_df DataFrame are "Hotel Name" and "Country".

map_plot_with_hotel = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    title='City Map with Nearest Hotel',
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],  # Include hotel name and country in the hover tooltip
    size='Humidity',
    frame_width=600,
    frame_height=400
)

# Display the map
map_plot_with_hotel


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)